In [ ]:
from bs4 import BeautifulSoup as bs
import random
import time
import pandas as pd
import undetected_chromedriver as uc
import time

In [2]:
browser = uc.Chrome()

In [3]:
url2= 'https://www.idealista.com/venta-viviendas/barcelona-barcelona/'

In [4]:
browser.get(url2)

In [5]:
browser.implicitly_wait(10)

In [6]:
# Una pagina completa

busqueda = 'barcelona'

busqueda = busqueda.replace(' ', '-')
x = 1
ids = []
df = pd.DataFrame(columns=['ID', 'Título', 'Enlace', 'Precio', 'Detalles', 'Descripción'])
df.to_csv("scraped_data.csv", mode="a", index=False, header=True)

while True:
    url = f'https://www.idealista.com/venta-viviendas/barcelona-barcelona/pagina-{x}.htm'

    browser.get(url)
    time.sleep(random.randint(10, 12))

    try:
        browser.find_element("xpath", '//*[@id="didomi-notice-agree-button"]').click()
    except:
        pass

    html = browser.page_source
    soup = bs(html, 'html.parser')

    # Check if the current page number matches the one displayed in pagination
    try:
        pagina_actual = int(soup.find('main', {'class': 'listing-items'}).find('div', {'class': 'pagination'}).find('li', {'class': 'selected'}).text)
    except AttributeError:
        print("Error: Could not find the pagination element.")
        break

    # Fetch the articles if the current page matches
    if x == pagina_actual:
        articles = soup.find('main', {'class': 'listing-items'}).find_all('article')
    else:
        break

    if not articles:
        print("No more apartments found.")
        break  # Stop if no articles are found

    for article in articles:
        id_muebles = article.get('data-element-id')

        # Título y enlace
        link = article.find('a', {'class': 'item-link'})
        title = link.get_text(strip=True) if link else "No disponible"
        href = link['href'] if link else "No disponible"
        full_link = f"https://www.idealista.com{href}" if href != "No disponible" else href
        
        # Precio
        price = article.find('span', {'class': 'item-price'})
        price_text = price.get_text(strip=True) if price else "No disponible"

        # Detalles adicionales
        details = article.find('div', {'class': 'item-detail-char'})
        details_text = details.get_text(" | ", strip=True) if details else "No disponible"

        # Descripción
        description = article.find('p', {'class': 'ellipsis'})
        description_text = description.get_text(strip=True) if description else "No disponible"

        # Añadir los datos al DataFrame
        new_row = pd.DataFrame({
            'ID': [id_muebles],  # Wrap the scalar in a list
            'Título': [title],
            'Enlace': [full_link],
            'Precio': [price_text],
            'Detalles': [details_text],
            'Descripción': [description_text]
        })
        print(new_row)
        
        df = pd.concat([df, new_row], ignore_index=True)
        
        # Escribir en el CSV después de cada artículo
        df.to_csv("scraped_data.csv", mode="a", index=False, header=False)
        
        # Pausa aleatoria para evitar sobrecargar el servidor
        time.sleep(random.randint(1, 3))

    # Check for the 'next' page link to stop if there are no more pages
    next_page = soup.find('a', {'class': 'icon-arrow-right-after'})
    if next_page is None:
        print("No more pages to scrape.")
        break  # Stop if there's no 'next' page link

    # Move to the next page
    x += 1

df


          ID                                       Título  \
0  106476051  Piso en paseo de Colom, El Gòtic, Barcelona   

                                          Enlace      Precio  \
0  https://www.idealista.com/inmueble/106476051/  1.275.000€   

                                            Detalles  \
0  2 hab. | 131 m² | Planta 2ª exterior con ascensor   

                                         Descripción  
0  Exclusivo piso de 131 m² en el Barrio Gótico d...  
          ID                                          Título  \
0  106475860  Piso en calle de la Mercè, El Gòtic, Barcelona   

                                          Enlace    Precio  \
0  https://www.idealista.com/inmueble/106475860/  765.000€   

                                            Detalles  \
0  2 hab. | 110 m² | Entreplanta exterior con asc...   

                                         Descripción  
0  Apartamento de lujo en Passeig de Colom con vi...  
          ID                                    

,ID,Título,Enlace,Precio,Detalles,Descripción
0,106476051,"Piso en paseo de Colom, El Gòtic, Barcelona",https://www.idealista.com/inmueble/106476051/,1.275.000€,2 hab. | 131 m² | Planta 2ª exterior con ascensor,Exclusivo piso de 131 m² en el Barrio Gótico d...
1,106475860,"Piso en calle de la Mercè, El Gòtic, Barcelona",https://www.idealista.com/inmueble/106475860/,765.000€,2 hab. | 110 m² | Entreplanta exterior con asc...,Apartamento de lujo en Passeig de Colom con vi...
2,103930029,"Ático en Gran Via de les Corts Catalanes, L'An...",https://www.idealista.com/inmueble/103930029/,750.000€,2 hab. | 97 m² | Planta 8ª exterior con ascensor,Este maravilloso ático de 87 m2 construidos in...
3,None,No disponible,No disponible,No disponible,No disponible,No disponible
4,105473433,"Piso en calle de València, La Dreta de l'Eixam...",https://www.idealista.com/inmueble/105473433/,2.450.000€,4 hab. | 253 m² | Planta 1ª exterior con ascensor,Esta increíble vivienda reformada a estrenar s...
...,...,...,...,...,...,...
1915,101127298,"Piso en El Poble Sec - Parc de Montjuïc, Barce...",https://www.idealista.com/inmueble/101127298/,292.000€,4 hab. | 90 m² | Planta 1ª sin ascensor,"Piso en Barcelona, perfecto para familias. El ..."
1916,105967284,"Piso en calle de la Junta de Comerç, El Raval,...",https://www.idealista.com/inmueble/105967284/,215.000€,3 hab. | 70 m² | Entreplanta con ascensor,Te presentamos un piso en venta ubicado en una...
1917,106286767,"Piso en La Sagrera, Barcelona",https://www.idealista.com/inmueble/106286767/,285.000€,3 hab. | 87 m² | Planta 4ª exterior con ascensor,Este piso de 3 habitaciones (dos dobles) ofrec...
1918,106436932,"Piso en Can Baró, Barcelona",https://www.idealista.com/inmueble/106436932/,579.000€,4 hab. | 160 m² | sin ascensor,Ático Dúplex con Vistas Panorámicas y Tres Ter...


In [8]:
len(df["ID"].unique())

1666

In [10]:
df.to_csv("data/data.csv")